In [ ]:
# 用單個模型同時執行兩個分類任務:
#   age 分成8個classes
#   gender 分成2個classes
# mlp 每個全連接層的unit個數: 128 -- 8
#                   \_ 2
# trainning: 
#   改用generator產生資料給fit_generator
#   class_weight
#   random_state
#   callback: EarlyStop, model.save

IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 20
DROP_RATE = 0.4
model_folder_path = 'drive/My Drive/Tibame_AIoT_Project/face'
img_folder_path = 'drive/My Drive/Tibame_AIoT_Project/Datasets/cleandataset'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [ ]:
# to measure execution time
!pip install ipython-autotime
%load_ext autotime

In [ ]:
! nvidia-smi

In [ ]:
!pip install mtcnn

In [ ]:
!pip install tensorflow==2.2.0

In [ ]:
import scipy.io
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

import tensorflow as tf
from tensorflow import keras
#import keras
#from keras.preprocessing.image import load_img
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.layers import Dense, Activation, Dropout, Flatten, Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Activation
from keras.layers import Conv2D, AveragePooling2D, BatchNormalization
from keras.models import Model, Sequential
from keras.regularizers import l1,l2,l1_l2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras import metrics

import matplotlib.pyplot as plt
%matplotlib inline

from keras.models import load_model
import cv2
from glob import glob
import os
from mtcnn import MTCNN
import numpy as np

In [ ]:
print(tf.__version__)
print(keras.__version__)

In [ ]:
# 資料集由csv檔案讀入, 也可新增其他的csv檔案來擴充資料
# df = pd.read_csv('drive/My Drive/Tibame_AIoT_Project/Datasets/資料集_IMDB-Wiki/age_gender_wiki.csv')
# df_under10 = pd.read_csv('drive/My Drive/Tibame_AIoT_Project/Datasets/資料集_IMDB-Wiki/age_gender_imdb_under10.csv')
# df_over70 = pd.read_csv('drive/My Drive/Tibame_AIoT_Project/Datasets/資料集_IMDB-Wiki/age_gender_imdb_over70.csv')
# df = pd.concat([df, df_under10, df_over70])

In [ ]:
# cleandata: 清除wiki資料集原本的一些年齡標註錯誤
df = pd.read_csv(os.path.join(img_folder_path, 'cleandata.csv'))

In [ ]:
#some guys seem to be greater than 100. some of these are paintings. remove these old guys
df = df[df['age'] <= 100]
 
#some guys seem to be unborn in the data set
df = df[df['age'] > 0]


In [ ]:
# 每10歲分一類,70歲以上歸為同一類,共8類
df['age_grp'] = pd.cut(df['age'], bins=[0,10,20,30,40,50,60,70,110], right=False)
le = LabelEncoder()
le.fit(df['age_grp'].astype('str'))
df['age_cls'] = le.transform(df['age_grp'].astype('str'))
df

In [ ]:
df['age_cls'].value_counts().sort_index()

In [ ]:
histogram_age = df['age_cls'].hist(bins=df['age_cls'].nunique())

In [ ]:
histogram_gender = df['gender'].hist(bins=df['gender'].nunique())

In [ ]:
s = df.groupby(['gender'])['age_grp'].value_counts()
print(s.index)
print(s.values)
s = s.sort_index()
s

In [ ]:
#因資料不平均, 且多輸出分類器不能用class_weight, 所以只用部分資料來訓練
FULL_DATA = 0
per_cls_trn = 600
# class:     f0  f1  f2   f3  f4  f5   f6  f7  m0  m1  m2   m3   m4   m5   m6   m7
start_idx = [0, 600, 600, 600, 0, 600, 600, 0, 0, 600, 600, 600, 600, 600, 600, 600]
end_idx = np.array(start_idx) + per_cls_trn
print(start_idx, end_idx)
per_cls_eval = 100
#用全部資料
#FULL_DATA = 1

In [ ]:
#先用少量資料比較不同模型:
#每個類別各取部分資料,用train_test_split來切train and test
df_f0 = df[(df['age_cls'] == 0) & (df['gender'] == 0)]
df_f1 = df[(df['age_cls'] == 1) & (df['gender'] == 0)]
df_f2 = df[(df['age_cls'] == 2) & (df['gender'] == 0)]
df_f3 = df[(df['age_cls'] == 3) & (df['gender'] == 0)]
df_f4 = df[(df['age_cls'] == 4) & (df['gender'] == 0)]
df_f5 = df[(df['age_cls'] == 5) & (df['gender'] == 0)]
df_f6 = df[(df['age_cls'] == 6) & (df['gender'] == 0)]
df_f7 = df[(df['age_cls'] == 7) & (df['gender'] == 0)]
df_m0 = df[(df['age_cls'] == 0) & (df['gender'] == 1)]
df_m1 = df[(df['age_cls'] == 1) & (df['gender'] == 1)]
df_m2 = df[(df['age_cls'] == 2) & (df['gender'] == 1)]
df_m3 = df[(df['age_cls'] == 3) & (df['gender'] == 1)]
df_m4 = df[(df['age_cls'] == 4) & (df['gender'] == 1)]
df_m5 = df[(df['age_cls'] == 5) & (df['gender'] == 1)]
df_m6 = df[(df['age_cls'] == 6) & (df['gender'] == 1)]
df_m7 = df[(df['age_cls'] == 7) & (df['gender'] == 1)]
# train and val data
if FULL_DATA == 1:
    #每個類別保留最後per_cls_eval筆資料作為evaluate用
    train_df = pd.concat([
        df_f0[:-per_cls_eval], df_f1[:-per_cls_eval], df_f2[:-per_cls_eval], df_f3[:-per_cls_eval], 
        df_f4[:-per_cls_eval], df_f5[:-per_cls_eval], df_f6[:-per_cls_eval], df_f7[:-per_cls_eval], 
        df_m0[:-per_cls_eval], df_m1[:-per_cls_eval], df_m2[:-per_cls_eval], df_m3[:-per_cls_eval], 
        df_m4[:-per_cls_eval], df_m5[:-per_cls_eval], df_m6[:-per_cls_eval], df_m7[:-per_cls_eval]         
        ])           
else:    
    #每個類別取同樣數量的資料來訓練
    train_df = pd.concat([
        df_f0[start_idx[0]:end_idx[0]], df_f1[start_idx[1]:end_idx[1]], df_f2[start_idx[2]:end_idx[2]], df_f3[start_idx[3]:end_idx[3]], 
        df_f4[start_idx[4]:end_idx[4]], df_f5[start_idx[5]:end_idx[5]], df_f6[start_idx[6]:end_idx[6]], df_f7[start_idx[7]:end_idx[7]], 
        df_m0[start_idx[8]:end_idx[8]], df_m1[start_idx[9]:end_idx[9]], df_m2[start_idx[10]:end_idx[10]], df_m3[start_idx[11]:end_idx[11]], 
        df_m4[start_idx[12]:end_idx[12]], df_m5[start_idx[13]:end_idx[13]], df_m6[start_idx[14]:end_idx[14]], df_m7[start_idx[15]:end_idx[15]]         
        ])
    
# evaluate data: 每個類別保留最後per_cls_eval筆資料作為evaluate用
eval_df = pd.concat([
        df_f0[-per_cls_eval:], df_f1[-per_cls_eval:], df_f2[-per_cls_eval:], df_f3[-per_cls_eval:], 
        df_f4[-per_cls_eval:], df_f5[-per_cls_eval:], df_f6[-per_cls_eval:], df_f7[-per_cls_eval:],
        df_m0[-per_cls_eval:], df_m1[-per_cls_eval:], df_m2[-per_cls_eval:], df_m3[-per_cls_eval:], 
        df_m4[-per_cls_eval:], df_m5[-per_cls_eval:], df_m6[-per_cls_eval:], df_m7[-per_cls_eval:]         
        ])
x_eval = np.array(eval_df['full_path'])
# 先把模型的兩個輸出的答案合併
y_eval = np.array(pd.concat([eval_df['age_cls'], eval_df['gender']], axis=1))
print("train:", len(train_df), "predict:", len(eval_df))

In [ ]:
# 處理答案 把它轉成one-hot (後面再做)
# y_train_category = to_categorical(df['age_cls'], num_classes=8)

# 2個輸出: age, gender
y_df = pd.concat([pd.DataFrame(train_df['age_cls']), pd.DataFrame(train_df['gender'])], axis=1)

# 切分訓練data
x_train, x_test, y_train, y_test = train_test_split(np.array(train_df['full_path']), np.array(y_df), test_size=0.2, random_state=0)

print(x_train[0], x_test[0], y_train[0], y_test[0])
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

In [ ]:
detector = MTCNN()
#feature_extractor = load_model(os.path.join(model_folder_path, 'facenet_keras.h5'))

In [ ]:
# VGGFace: https://github.com/rcmalli/keras-vggface
!pip install keras_vggface
!pip install keras_applications

from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
feature_extractor = VGGFace(model='vgg16', include_top=False, 
            input_shape=(224, 224, 3), pooling='avg')

In [ ]:
feature_extractor.summary()

In [ ]:
# 固定pre-train model的參數
for lyr in feature_extractor.layers:
    lyr.trainable = False

# BN
x = BatchNormalization()(feature_extractor.output)    
    
# MLP    
# x = Flatten()(x)

# if DROP_RATE != 0:
#     x = Dropout(DROP_RATE)(x)
# x = Dense(units=512, activation='relu', kernel_regularizer='l1_l2')(x)

if DROP_RATE != 0:
    x = Dropout(DROP_RATE)(x)
x = Dense(units=128, activation='relu', kernel_regularizer='l1_l2')(x)

if DROP_RATE != 0:
    x = Dropout(DROP_RATE)(x)
age = Dense(units=8, activation='softmax', name='age', kernel_regularizer='l1_l2')(x)
gender = Dense(units=2, activation='softmax', name='gender', kernel_regularizer='l1_l2')(x)

inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
# 2個輸出: age, gender
age_gender_model = Model(inputs=feature_extractor.input, outputs=[age, gender])   
age_gender_model.summary()

In [ ]:
from keras.utils import plot_model
plot_model(age_gender_model, show_shapes=True)

In [ ]:
age_gender_model.compile(loss=["categorical_crossentropy","categorical_crossentropy"], 
                  optimizer='adam', metrics=[{'age':'accuracy'},{'gender':'accuracy'}]) # 2個輸出: age, gender

age_gender_model = load_model(os.path.join(model_folder_path,'21-3_resnet_mlp128_bs64_ep9_0.4746_0.9017.h5'))

In [ ]:
# # 資料預處理 for facenet?
# # Standardization
# def preprocess(imgs): 
#     for i in range(imgs.shape[0]):
#         # standardization
#         img = imgs[i]
#         mean, std = img.mean(), img.std()
#         img = (img - mean) / std
#         imgs[i] = img
#     return imgs
# # Normalization
# def normalize(img):
#     return img / 255.

# # -1 <= x <= 1
# def preprocess_1(imgs):
#     x = np.array(imgs, dtype = float)
#     x /= 127.5
#     x -= 1.
#     return x    

In [ ]:
import imgaug.augmenters as iaa
import imgaug as ia
# augmentation
seq = iaa.Sequential([
    #iaa.Crop(px=(0, 16)), # crop images from each side by 0 to 16px (randomly chosen)
    iaa.Fliplr(0.5), # horizontally flip 50% of the images
    iaa.GaussianBlur(sigma=(0, 3.0)), # blur images with a sigma of 0 to 3.0
    #iaa.Flipud(0.5),
    iaa.Affine(
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}
        # rotate=(-45, 45),
        # mode=ia.ALL, # edge, reflect, symmetric, warp, constant
        # shear=(-16,16)
    )
])

time: 10.4 ms


In [ ]:
# detect face
def detect_faces(img):
    face_imgs = []

    results = detector.detect_faces(img)
    # extract the bounding box from the first face
    # print('# of faces: ', len(results))
    for i in range(len(results)):
        x1, y1, width, height = results[i]['box']
        x2, y2 = x1 + width, y1 + height
        patch = img[y1:y2, x1:x2] # crop face
        face_imgs.append(patch)
     
    return face_imgs

In [ ]:
from tensorflow.keras.utils import Sequence
class DataGenerator(Sequence):
    """
    Generates data for Keras
    ref: https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly
    """
    def __init__(self,
                 paths,
                 y_cls,
                 batch_size,
                 #num_classes,
                 shuffle=False,
                 augment=False):
        self.paths = paths
        self.y_cls = y_cls
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.augment = augment
        #self.num_classes = num_classes
        self.indexes = np.arange(len(self.paths))
        #self.class_map = {'0':0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7}
        self.on_epoch_end()

    def __len__(self):
        'number of batches per epoch'
        return int(np.ceil(len(self.paths) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'

        # Generate indexes of the batch
        idxs = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]

        # Find list of IDs
        batch_paths = [self.paths[i] for i in idxs]
        batch_y = [self.y_cls[i] for i in idxs]

        # Generate data
        X, y = self.__data_generation(batch_paths, batch_y)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __data_generation(self, paths, y_cls):
        """
        Generates data containing batch_size samples
        """
        # X = np.empty((len(paths), IMG_SIZE, IMG_SIZE, 3), dtype=np.float32)
        # y = np.empty((len(paths), self.num_classes), dtype=np.float32)

        x_ori, x_norm, y_age, y_gender = [], [], [], []

        for i, path in enumerate(paths):
            #print("idx:", i, "cls:", y_cls[i], path)
        
            # 讀取圖片,切下臉的部分,並使用借來的模型的預處理方式來作預處理 
            try:          
                img = cv2.imread(os.path.join(img_folder_path,path))[:,:,::-1]
            except:
                print('imread failed:', path)
                continue 

            if self.augment:
                img = seq.augment_image(img)



            faces = detect_faces(img)
            if len(faces) == 0 or faces[0].shape[0] == 0 or faces[0].shape[1] == 0:
                print(' No face')
                continue
            #print(faces[0].shape)   
            img_crop = cv2.resize(faces[0], (IMG_SIZE, IMG_SIZE))

            # 使用借來的模型的預處理方式來作預處理
            img_pre = preprocess_input(np.array(img_crop,dtype=float))
            
            # 把原圖留下來
            x_ori.append(img)
            x_norm.append(img_pre)
            y_age.append(y_cls[i][0])
            y_gender.append(y_cls[i][1])

            

        # print("len(image_data)",len(x_ori))
        # plt.figure(figsize=(10, 40))
        # for j,m in enumerate(x_ori):
        #     plt.subplot(1, BATCH_SIZE, (j%BATCH_SIZE)+1)
        #     plt.title("idx:{} y_cls:{}".format(i_batch+j, y_cls[i_batch+j]))
        #     plt.axis("off")
        #     plt.imshow(m)
        # plt.show() 

        
        # 2個輸出: age, gender  
        # print(type(y_age), len(y_age), y_age[:8])
        # print(type(y_gender), len(y_gender), y_gender[:8])
        y_age_category = to_categorical(y_age, num_classes=8) 
        y_gender_category = to_categorical(y_gender, num_classes=2) 
        # print(y_age_category)
        # print(y_gender_category)
        x_input = {'input_4':np.array(x_norm)}
        y_category = {'age':np.array(y_age_category), 'gender':np.array(y_gender_category)}
        # print(type(np.array(x_norm)), np.array(x_norm).shape)
        # print(type(y_category), np.array(y_age_category), np.array(y_gender_category))

        #yield x_input, y_category
        return x_input, y_category

In [ ]:
# 用generator產生資料
generator_train = DataGenerator(x_train, y_train, batch_size=BATCH_SIZE, augment=True)
generator_test = DataGenerator(x_test, y_test, batch_size=BATCH_SIZE)
#type(generator_train)

In [ ]:
# if FULL_DATA == 1:
#     age_weights = {0:12., 1:5., 2:1., 3:2., 4:3., 5:4., 6:6., 7:3.}
# else:    
#     # for temp
#     age_weights = {0:1., 1:1., 2:1., 3:1., 4:1., 5:1., 6:1., 7:1.}

data_count = np.unique(np.argmax(y_train, axis=-1), return_counts=True)[1]
data_count
num_classes=8
age_weights = (1/data_count)*np.sum(data_count)/num_classes
class_weight = {i: w for i, w in enumerate(age_weights)}
print('class_weight', class_weight)

In [ ]:
# fit_generator
checkpoint = ModelCheckpoint(os.path.join(model_folder_path,"21-4_resnet_mlp128_bs64_aug_ep{epoch}_{val_age_accuracy:.4f}_{val_gender_accuracy:.4f}.h5"), 
               save_best_only=False, save_weights_only=False)   #Defaults: save_freq='epoch', save_weights_only=False
earlystop = EarlyStopping(patience=5, restore_best_weights=True)
#logs = age_gender_model.fit_generator(
logs = age_gender_model.fit( 
        generator_train,
        #steps_per_epoch=len(x_train)//BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=generator_test,
        #validation_steps=len(x_test)//BATCH_SIZE,
        #class_weight = class_weight,    # `class_weight` is only supported for Models with a single output.
        callbacks=[checkpoint, earlystop] 
        )

In [ ]:
age_gender_model.save(os.path.join(model_folder_path,'21-4_resnet_mlp128_bs64_aug_save.h5'))

In [ ]:
model = load_model(os.path.join(model_folder_path,"21-4_resnet_mlp128_bs64_aug_save.h5"))

In [ ]:
history = logs.history
history

In [ ]:
plt.plot(history['age_accuracy'])
plt.plot(history['val_age_accuracy'])
plt.legend(['age_accuracy', 'val_age_accuracy'])
plt.title('age_accuracy')
plt.show()

In [ ]:
plt.plot(history['gender_accuracy'])
plt.plot(history['val_gender_accuracy'])
plt.legend(['gender_accuracy', 'val_gender_accuracy'])
plt.title('gender_accuracy')

In [ ]:
plt.plot(history['age_loss'])
plt.plot(history['val_age_loss'])
plt.legend(['age_loss', 'val_age_loss'])
plt.title('age_loss')

In [ ]:
plt.plot(history['gender_loss'])
plt.plot(history['val_gender_loss'])
plt.legend(['gender_loss', 'val_gender_loss'])
plt.title('gender_loss')

In [ ]:
!pip install keras2onnx

In [ ]:
# convert to onnx model
import keras2onnx
onnx_model = keras2onnx.convert_keras(age_gender_model, age_gender_model.name)

In [ ]:
keras2onnx.save_model(onnx_model, os.path.join(model_folder_path,'23-3_resnet_mlp512-128.onnx'))

In [ ]:
# 取得要預測的圖片並做預處理
def get_preprocess_images(data_paths, y_data, batch_size=BATCH_SIZE):
    n = len(data_paths)
    # i = 0
    # data_paths = data_paths
    
    #while i < n:    
    x_ori, x_norm, y_age, y_gender = [], [], [], []
    #i_batch = i
    for idx in range(batch_size):
        path = data_paths[idx]
        #print("n:", n, "idx:", i, "cls:", y_data[i], path)
    
        # 讀取圖片,切下臉的部分,並使用借來的模型的預處理方式來作預處理 
        try:          
            img = cv2.imread(os.path.join(img_folder_path,path))[:,:,::-1]
        except:
            print('imread failed:', path)
            idx = idx + 1
            continue                   
        
        faces = detect_faces(img)
        if len(faces) == 0 or faces[0].shape[0] == 0 or faces[0].shape[1] == 0:
            print('No face')
            idx = idx + 1
            continue   
        # print(faces[0].shape)    
        img_crop = cv2.resize(faces[0], (IMG_SIZE, IMG_SIZE))
        

        # 使用借來的模型的預處理方式來作預處理
        img_pre = preprocess_input(np.array(img_crop, dtype=float))

        # 把原圖留下來
        x_ori.append(img)
        x_norm.append(img_pre)
        if len(y_data) != 0:
            y_age.append(y_data[idx][0])
            y_gender.append(y_data[idx][1])
        
        idx = idx + 1


    # print("len(image_data)",len(x_ori))
    # plt.figure(figsize=(10, 40))
    # for j,m in enumerate(x_ori):
    #     plt.subplot(1, BATCH_SIZE, (j%BATCH_SIZE)+1)
    #     plt.title("idx:{} y_data:{}".format(i_batch+j, y_data[i_batch+j]))
    #     plt.axis("off")
    #     plt.imshow(m)
    # plt.show() 

    
    # 2個輸出: age, gender  
    # print(type(y_age), len(y_age), y_age[:8])
    # print(type(y_gender), len(y_gender), y_gender[:8])
    if len(y_data) != 0:
        y_age_category = to_categorical(y_age, num_classes=8) 
        y_gender_category = to_categorical(y_gender, num_classes=2) 
        y_category = {'age':np.array(y_age_category), 'gender':np.array(y_gender_category)}
    else:
        y_category = []

    # print(type(np.array(x_norm)), np.array(x_norm).shape)
    # print(type(y_category), np.array(y_age_category), np.array(y_gender_category))

    return np.array(x_ori), np.array(x_norm), y_category
    #print('while end', i, n)

In [ ]:
# evaluate
#
# 從保留作為evaluate用的資料,用generator產生資料 to predict
x_ori, x_input, y_category = get_preprocess_images(x_eval, y_eval, batch_size=len(x_eval))

# 取出圖片資料與正確答案
x_eval_data, y_true_age, y_true_gender = [], [], []
for i,x in enumerate(x_input):
    # print("x_eval_data:", len(list(x_dict['input_4'])))
    x_eval_data.append(x)
    # print("y_true_age:", y_dict['age'].argmax(axis=-1))
    # print("y_true_gender:", y_dict['gender'].argmax(axis=-1))    
    y_true_age.append( (list(y_category['age'])[i].argmax(axis=-1)) )
    y_true_gender.append( (list(y_category['gender'])[i].argmax(axis=-1)) )

# print("-------------------------")
print("x_eval_data:", type(x_eval_data), "np.array:", np.array(x_eval_data).shape, x_eval[:8])
print("y_true_age:", y_true_age)
print("y_true_gender:", y_true_gender)

In [ ]:
# predict
pre = age_gender_model.predict(np.array(x_eval_data))
#pre[0] is predicted probabilities for age
#pre[1] is predicted probabilities for gender
pred_age = pre[0].argmax(axis=-1)
pred_gender = pre[1].argmax(axis=-1)
print("predict age:",pred_age)
print("predict gender:",pred_gender)

In [ ]:
len(pred_age), len(pred_gender)

In [ ]:
from sklearn.metrics import classification_report
print(np.array(y_true_age).shape, np.array(pred_age).shape, np.array(y_true_gender).shape, np.array(pred_gender).shape)
age_target_names = [str(i) for i in range(8)]
gender_target_names = [str(i) for i in range(2)]
print(classification_report(np.array(y_true_age), np.array(pred_age), target_names=age_target_names))
print(classification_report(np.array(y_true_gender), np.array(pred_gender), target_names=gender_target_names))

In [ ]:
from sklearn.metrics import confusion_matrix
pd.DataFrame(confusion_matrix(y_true_age, pred_age),
            index=["{}(真實)".format(i) for i in range(8)],
            columns=["{}(預測)".format(i) for i in range(8)] 
            )

In [ ]:
pd.DataFrame(confusion_matrix(y_true_gender, pred_gender),
            index=["{}(真實)".format(i) for i in range(2)],
            columns=["{}(預測)".format(i) for i in range(2)] 
            )